# Deep Learning Assignment 1
## Submission by Akhil Sudhakaran 2021MT12054



In [1]:
import gc
import os
import re
import glob
import keras
import numpy  as np
import pandas as pd
import tensorflow as tf
import plotly.express as px
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from sklearn.metrics                        import multilabel_confusion_matrix
from sklearn.model_selection                import train_test_split
from tensorflow.keras.models                import Sequential
from tensorflow.keras.layers                import Dense, Dropout, GlobalMaxPooling2D
from tensorflow.keras.callbacks             import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, CSVLogger
from tensorflow.keras.optimizers            import Adam, SGD
from tensorflow.keras.applications          import InceptionV3, MobileNetV2, MobileNetV3Large
from tensorflow.keras.preprocessing         import image
from tensorflow.keras.preprocessing.image   import ImageDataGenerator

## Creating training and test dataframes

In [2]:
temp = []
for dirname, _, filenames in os.walk('/kaggle/input/deep-learning-sszg529-bits-pilani/train_data/train_data/'):
    for filename in filenames:
        filePath = os.path.join(dirname, filename)
        if(dirname == '/kaggle/input/deep-learning-sszg529-bits-pilani/train_data/train_data/Happiness'):
            temp.append({'FilePath': filePath, 'Class': "Happiness"})
        elif(dirname == '/kaggle/input/deep-learning-sszg529-bits-pilani/train_data/train_data/Other'):
            temp.append({'FilePath': filePath, 'Class': "Other"})
        elif(dirname == '/kaggle/input/deep-learning-sszg529-bits-pilani/train_data/train_data/Anger'):
            temp.append({'FilePath': filePath, 'Class': "Anger"})
        elif(dirname == '/kaggle/input/deep-learning-sszg529-bits-pilani/train_data/train_data/Contempt'):
            temp.append({'FilePath': filePath, 'Class': "Contempt"})
        elif(dirname == '/kaggle/input/deep-learning-sszg529-bits-pilani/train_data/train_data/Surprise'):
            temp.append({'FilePath': filePath, 'Class': "Surprise"})

Data = pd.DataFrame(temp).sample(frac = 1).reset_index()
Data

,index,FilePath,Class
0,2551,/kaggle/input/deep-learning-sszg529-bits-pilan...,Other
1,3391,/kaggle/input/deep-learning-sszg529-bits-pilan...,Happiness
2,1777,/kaggle/input/deep-learning-sszg529-bits-pilan...,Other
3,5987,/kaggle/input/deep-learning-sszg529-bits-pilan...,Anger
4,1055,/kaggle/input/deep-learning-sszg529-bits-pilan...,Other
...,...,...,...
8176,8021,/kaggle/input/deep-learning-sszg529-bits-pilan...,Anger
8177,2252,/kaggle/input/deep-learning-sszg529-bits-pilan...,Other
8178,4910,/kaggle/input/deep-learning-sszg529-bits-pilan...,Anger
8179,8140,/kaggle/input/deep-learning-sszg529-bits-pilan...,Anger


## Training data class distribution

In [3]:
label_count_df = pd.DataFrame(Data['Class'].value_counts()).reset_index()
label_count_df

,index,Class
0,Anger,3362
1,Other,1702
2,Happiness,1550
3,Surprise,850
4,Contempt,717


In [4]:
temp = []
for dirname, _, filenames in os.walk('/kaggle/input/deep-learning-sszg529-bits-pilani/test_data/test_data/'):
    for filename in filenames:
        temp.append({'FilePath': os.path.join(dirname, filename)})

predictionData = pd.DataFrame(temp)
predictionData

,FilePath
0,/kaggle/input/deep-learning-sszg529-bits-pilan...
1,/kaggle/input/deep-learning-sszg529-bits-pilan...
2,/kaggle/input/deep-learning-sszg529-bits-pilan...
3,/kaggle/input/deep-learning-sszg529-bits-pilan...
4,/kaggle/input/deep-learning-sszg529-bits-pilan...
...,...
1938,/kaggle/input/deep-learning-sszg529-bits-pilan...
1939,/kaggle/input/deep-learning-sszg529-bits-pilan...
1940,/kaggle/input/deep-learning-sszg529-bits-pilan...
1941,/kaggle/input/deep-learning-sszg529-bits-pilan...


## Configurable params

In [5]:
batch_size = 32
image_size_x = 229
image_size_y = 229
epochs     = 50

In [6]:
train, test = train_test_split(
                    Data,
                    test_size = 0.2
                )

In [7]:
train_generator = ImageDataGenerator(
                        rescale     = 1./255,
                        zoom_range  = 0.1,
                        fill_mode='nearest',
                        horizontal_flip = True
                    )

test_generator = ImageDataGenerator(rescale=1./255)

In [8]:
train_gen = train_generator.flow_from_dataframe(
                                    dataframe   = train,
                                    class_mode  = 'categorical',
                                    x_col       = 'FilePath',
                                    y_col       = 'Class',
                                    shuffle     = True,
                                    weight_col  = None,
                                    batch_size  = batch_size,
                                    target_size = (image_size_x, image_size_y),
                                    seed        = None
                                )

test_gen  = test_generator.flow_from_dataframe(
                                    dataframe   = test,
                                    class_mode  = 'categorical',
                                    x_col       = 'FilePath',
                                    y_col       = 'Class',
                                    shuffle     = False,
                                    batch_size  = batch_size,
                                    weight_col  = None,
                                    target_size = (image_size_x, image_size_y),
                                    seed        = None
                                )

Found 6544 validated image filenames belonging to 5 classes.
Found 1637 validated image filenames belonging to 5 classes.


## Model
The model looks something like this:

**[Resized_Input_Image] -> [InceptionV3_Convolution_Layer_with_imagenet] -> [2D_Max_Pooling] -> [Fully_Connected_Classifier_with_Softmax]**

Notes on model training:

Dropout to reduce overfitting since earlier runs on training sample gave less accuracy
Adam optimizer for better convergence
Categorical crossentropy since we are doing a multiclass classification
Using GPU to accelerate since CPU was too slow

In [9]:
conv_net = InceptionV3(input_shape=(image_size_x, image_size_y, 3), weights='imagenet', include_top=False)
# Freezing layers
for layer in conv_net.layers[:249]:
    layer.trainable = False
for layer in conv_net.layers[249:]:
    layer.trainable = True

model = Sequential()
model.add(conv_net)
model.add(GlobalMaxPooling2D())
model.add(Dropout(0.3))
model.add(Dense(1024, activation='relu'))
model.add(Dense(5,activation='softmax'))
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

2022-10-19 18:47:32.660920: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


87924736/87910968 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_max_pooling2d (Global (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 5125      
Total params: 23,906,085
Trainable params: 13,218,181
Non-trainable params: 10,687,904
_________________________________________________________________


## Model Training

In [10]:
training_hist = model.fit(
                    train_gen,
                    epochs = epochs,
                )

2022-10-19 18:47:37.722678: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
205/205 [==============================] - 537s 3s/step - loss: 0.7675 - accuracy: 0.7286
Epoch 2/50
205/205 [==============================] - 502s 2s/step - loss: 0.3883 - accuracy: 0.8524
Epoch 3/50
205/205 [==============================] - 501s 2s/step - loss: 0.2586 - accuracy: 0.8972
Epoch 4/50
205/205 [==============================] - 503s 2s/step - loss: 0.2137 - accuracy: 0.9213
Epoch 5/50
205/205 [==============================] - 503s 2s/step - loss: 0.1873 - accuracy: 0.9306
Epoch 6/50
205/205 [==============================] - 512s 2s/step - loss: 0.1339 - accuracy: 0.9496
Epoch 7/50
205/205 [==============================] - 516s 3s/step - loss: 0.1228 - accuracy: 0.9531
Epoch 8/50
205/205 [==============================] - 505s 2s/step - loss: 0.1085 - accuracy: 0.9619
Epoch 9/50
205/205 [==============================] - 504s 2s/step - loss: 0.1215 - accuracy: 0.9539
Epoch 10/50
205/205 [==============================] - 514s 3s/step - loss: 0.1294 - accura

#### Evaluating model on validation data

In [11]:
results = model.evaluate_generator(test_gen)
print("Training data categorical accuracy = {}%".format(round(results[1]*100,2)))

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Training data categorical accuracy = 99.45%


#### Consufsion matrix on validation data predictions

In [12]:
testSetPred = model.predict_generator(test_gen)
confusion_matrix = np.sum(multilabel_confusion_matrix(pd.get_dummies(test['Class']), testSetPred >= 0.5),axis=0)
fig = px.imshow(confusion_matrix, color_continuous_scale = px.colors.sequential.Blugrn)
fig.update_layout(title_text='Confusion Matrix', title_x=0.5)
fig.show()

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


## Predict for a single image

In [13]:
# img = image.load_img('/kaggle/input/deep-learning-sszg529-bits-pilani/test_data/test_data/test_1080.jpg', target_size=(image_size_x, image_size_y))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x /= 255
# prob = model.predict(x)
# pred = prob.argmax(axis=-1)
# predClass = list(classes.keys())[list(classes.values()).index(pred)]
# print(predClass)

## Predict all test data
#### Creating the prediction result data frame

In [14]:
classes = train_gen.class_indices
testDf = pd.read_csv('/kaggle/input/deep-learning-sszg529-bits-pilani/test.csv')
temp = []
for dirname, _, filenames in os.walk('/kaggle/input/deep-learning-sszg529-bits-pilani/test_data/test_data/'):
    for filename in filenames:
        filePath = os.path.join(dirname, filename)
        img = image.load_img(filePath, target_size=(image_size_x, image_size_y))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x /= 255
        prob = model.predict(x)
        pred = prob.argmax(axis=-1)
        predClass = list(classes.keys())[list(classes.values()).index(pred)]
        if(predClass == 'Happiness'):
            temp.append({'Id': testDf.loc[testDf['Filename'] == filename, 'Id'].values[0], 'Class': 1})
        elif(predClass == 'Other'):
            temp.append({'Id': testDf.loc[testDf['Filename'] == filename, 'Id'].values[0], 'Class': 2})
        elif(predClass == 'Anger'):
            temp.append({'Id': testDf.loc[testDf['Filename'] == filename, 'Id'].values[0], 'Class': 3})
        elif(predClass == 'Contempt'):
            temp.append({'Id': testDf.loc[testDf['Filename'] == filename, 'Id'].values[0], 'Class': 4})
        elif(predClass == 'Surprise'):
            temp.append({'Id': testDf.loc[testDf['Filename'] == filename, 'Id'].values[0], 'Class': 5})
predDf = pd.DataFrame(temp)

#### Save final predictions to CSV

In [15]:
predDf.to_csv('submission.csv', index=False)